In [ ]:
!pip install tensorflow-gpu==2.10.0 


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 38.3 MB/s eta 0:00:00m eta 0:00:010:00:01


In [4]:
! pip install ffmpeg-python pillow


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
!pip install "numpy<2"


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/4b/d7/ecf66c1cd12dc28b4040b15ab4d17b773b87fa9d29ca16125de01adb36cd/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


In [4]:
!pip list

Package                      Version
---------------------------- --------------
absl-py                      2.1.0
anyio                        4.4.0
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.4.1
astunparse                   1.6.3
async-lru                    2.0.4
attrs                        24.2.0
babel                        2.16.0
beautifulsoup4               4.12.3
bleach                       6.1.0
cachetools                   5.5.0
certifi                      2024.8.30
cffi                         1.17.1
charset-normalizer           3.3.2
comm                         0.2.2
debugpy                      1.8.5
decorator                    5.1.1
defusedxml                   0.7.1
exceptiongroup               1.2.2
executing                    2.1.0
fastjsonschema               2.20.0
ffmpeg-python                0.2.0
flatbuffers                  24.3.25
fqdn                         1.

In [1]:
import tensorflow as tf

2024-09-07 18:01:30.032292: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-07 18:01:30.150967: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-07 18:01:30.156358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-09-07 18:01:30.156369: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

### 移动视频符合要求目录
把 video_org下面的mp4文件，移动到 video_org_test下面 一个视频一个文件夹，命名使用test_{index}/test_{index}.mp4



In [10]:
import os
import shutil

# 源文件夹和目标文件夹路径
source_dir = '/mnt/ceph/develop/jiawei/lora_dataset/video_org'
dest_dir = '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/'

# 获取所有的 mp4 文件
mp4_files = [f for f in os.listdir(source_dir) if f.endswith('.mp4')]

# 遍历每个文件并创建目标文件夹
for index, file_name in enumerate(mp4_files, 1):
    # 创建新的文件夹名称
    new_folder_name = f'test_{index}'
    new_folder_path = os.path.join(dest_dir, new_folder_name)
    os.makedirs(new_folder_path, exist_ok=True)
    
    # 移动并重命名文件
    src_file_path = os.path.join(source_dir, file_name)
    dest_file_path = os.path.join(new_folder_path, f'test_{index}.mp4')
    shutil.move(src_file_path, dest_file_path)

print("搞定，所有视频都整好了！💪")


搞定，所有视频都整好了！💪


In [13]:
from transnetv2 import TransNetV2
visualize = False
import os
import sys
import numpy as np

load_files = []
root_path = '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/'
for root, dirs, files in os.walk(root_path):
    # 如果你只想获取下一层的子目录，可以在这里筛选
    if root == root_path:
        # root_dir 下的直接子目录就是 dirs 中的项
        for dir in dirs:
            video_path=f'{root_path}/{dir}/{dir}.mp4'
            load_files.append(video_path)


print(f'load_files:{load_files}')
model = TransNetV2(None)



for file in load_files:
    if os.path.exists(file + ".predictions.txt") or os.path.exists(file + ".scenes.txt"):
        print(f"[TransNetV2] {file}.predictions.txt or {file}.scenes.txt already exists. "
              f"Skipping video {file}.", file=sys.stderr)
        continue

    video_frames, single_frame_predictions, all_frame_predictions = \
        model.predict_video(file)

    predictions = np.stack([single_frame_predictions, all_frame_predictions], 1)
    np.savetxt(file + ".predictions.txt", predictions, fmt="%.6f")

    scenes = model.predictions_to_scenes(single_frame_predictions)
    np.savetxt(file + ".scenes.txt", scenes, fmt="%d")

    if visualize:
        if os.path.exists(file + ".vis.png"):
            print(f"[TransNetV2] {file}.vis.png already exists. "
                  f"Skipping visualization of video {file}.", file=sys.stderr)
            continue

        pil_image = model.visualize_predictions(
            video_frames, predictions=(single_frame_predictions, all_frame_predictions))
        pil_image.save(file + ".vis.png")


load_files:['/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test1/test1.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test2/test2.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1/test_1.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_10/test_10.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_11/test_11.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_12/test_12.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_13/test_13.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_14/test_14.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_15/test_15.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_16/test_16.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_17/test_17.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_18/test_18.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_19/test_19.mp4

[TransNetV2] /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test1/test1.mp4.predictions.txt or /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test1/test1.mp4.scenes.txt already exists. Skipping video /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test1/test1.mp4.
[TransNetV2] /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test2/test2.mp4.predictions.txt or /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test2/test2.mp4.scenes.txt already exists. Skipping video /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test2/test2.mp4.


[TransNetV2] Extracting frames from /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1/test_1.mp4
[TransNetV2] Processing video frames 5378/5378
[TransNetV2] Extracting frames from /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_10/test_10.mp4
[TransNetV2] Processing video frames 316/316
[TransNetV2] Extracting frames from /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_11/test_11.mp4
[TransNetV2] Processing video frames 299/299
[TransNetV2] Extracting frames from /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_12/test_12.mp4
[TransNetV2] Processing video frames 389/389
[TransNetV2] Extracting frames from /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_13/test_13.mp4
[TransNetV2] Processing video frames 241/241
[TransNetV2] Extracting frames from /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_14/test_14.mp4
[TransNetV2] Processing video frames 169/169
[TransNetV2] Extracting frames from /mnt/ceph/develop/jiawei/lora_data